In [123]:
#import libraries
import pandas as pd
import datetime as dt
import time 

Distance, market and tariff data are read.

In [124]:
dist = pd.read_csv("datathon_case_1/case_1_lu_distance.csv")

In [125]:
market = pd.read_csv("datathon_case_1/case_1_market_data.csv")

In [126]:
tariff = pd.read_csv("datathon_case_1/case_1_tariff_data.csv")

Data info:

In [83]:
dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121131944 entries, 0 to 121131943
Data columns (total 3 columns):
ORIGIN         object
DESTINATION    object
DISTANCE       float64
dtypes: float64(1), object(2)
memory usage: 2.7+ GB


In [84]:
market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719202 entries, 0 to 1719201
Data columns (total 4 columns):
SEASON         object
ORIGIN         object
DESTINATION    object
PSGR_COUNT     int64
dtypes: int64(1), object(3)
memory usage: 52.5+ MB


In [85]:
tariff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16990 entries, 0 to 16989
Data columns (total 11 columns):
SEASON              16990 non-null object
CARRIER             16990 non-null object
FLIGHT_NO           16976 non-null object
DEP_APT             16990 non-null object
ARR_APT             16990 non-null object
DEP_YMD_LMT         16990 non-null int64
DEP_TIME_LMT        16990 non-null int64
ARR_YMD_LMT         16990 non-null int64
ARR_TIME_LMT        16990 non-null int64
FLIGHT_DIRECTION    16990 non-null object
DISTANCE            16990 non-null float64
dtypes: float64(1), int64(4), object(6)
memory usage: 1.4+ MB


### Preprocessing

We combine DEP_YMD_LMT with  DEP_TIME_LMT and ARR_YMD_LMT with ARR_TIME_LMT into a datetime format and  make use of them while calculating connection time. (p.s. it is also useful for Case2)

In [127]:
tariff['DEP_YMD_LMT'] = pd.to_datetime(tariff['DEP_YMD_LMT'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
tariff['ARR_YMD_LMT'] = pd.to_datetime(tariff['ARR_YMD_LMT'], format='%Y%m%d').dt.strftime("%Y-%m-%d")

In [128]:
tariff['DEP_TIME_LMT'] = tariff['DEP_TIME_LMT'].astype(str)
tariff['ARR_TIME_LMT'] = tariff['ARR_TIME_LMT'].astype(str)

tariff['DEP_TIME_LMT'] = tariff['DEP_TIME_LMT'].apply(lambda x: '{0:0>4}'.format(x))
tariff['ARR_TIME_LMT'] = tariff['ARR_TIME_LMT'].apply(lambda x: '{0:0>4}'.format(x))

In [129]:
tariff['DEP_TIME_LMT'] = pd.to_datetime(tariff['DEP_TIME_LMT'], format='%H%M').dt.strftime("%H:%M")
tariff['ARR_TIME_LMT'] = pd.to_datetime(tariff['ARR_TIME_LMT'], format='%H%M').dt.strftime("%H:%M")

In [130]:
tariff["DEP_TIME"] = pd.to_datetime(tariff['DEP_YMD_LMT'].astype(str) + ' ' + tariff['DEP_TIME_LMT'].astype(str))
tariff["ARR_TIME"] = pd.to_datetime(tariff['ARR_YMD_LMT'].astype(str) + ' ' + tariff['ARR_TIME_LMT'].astype(str))

In [356]:
#checking the format
tariff.head()

,SEASON,CARRIER,FLIGHT_NO,DEP_APT,ARR_APT,DEP_YMD_LMT,DEP_TIME_LMT,ARR_YMD_LMT,ARR_TIME_LMT,FLIGHT_DIRECTION,DISTANCE,DEP_TIME,ARR_TIME
0,SUMMER,1K,KE,J74,I36,2018-08-20,18:25,2018-08-21,05:00,ID,7475.34,2018-08-20 18:25:00,2018-08-21 05:00:00
1,SUMMER,1K,A5,GRU,EZE,2018-08-24,18:05,2018-08-24,21:05,II,1710.75,2018-08-24 18:05:00,2018-08-24 21:05:00
2,SUMMER,1K,E5A,2L2,I36,2018-08-25,06:45,2018-08-25,09:50,ID,3952.66,2018-08-25 06:45:00,2018-08-25 09:50:00
3,SUMMER,1K,K0N,LED,I36,2018-08-26,15:15,2018-08-26,18:45,ID,2041.55,2018-08-26 15:15:00,2018-08-26 18:45:00
4,SUMMER,1K,AK6,32W,RUH,2018-08-22,21:25,2018-08-23,01:30,DI,2390.30,2018-08-22 21:25:00,2018-08-23 01:30:00


In [131]:
tariff['DEP_YMD_LMT'] = pd.to_datetime(tariff['DEP_YMD_LMT'])
tariff['ARR_YMD_LMT'] = pd.to_datetime(tariff['ARR_YMD_LMT'])

tariff['DEPARTURE_DAY'] = tariff['DEP_YMD_LMT'].dt.day_name()
tariff['ARRIVAL_DAY'] = tariff['ARR_YMD_LMT'].dt.day_name()

In [132]:
tariff = tariff.drop(columns=["DEP_YMD_LMT", "DEP_TIME_LMT", "ARR_YMD_LMT", "ARR_TIME_LMT"])

In [133]:
tariff.head(20)

,SEASON,CARRIER,FLIGHT_NO,DEP_APT,ARR_APT,FLIGHT_DIRECTION,DISTANCE,DEP_TIME,ARR_TIME,DEPARTURE_DAY,ARRIVAL_DAY
0,SUMMER,1K,KE,J74,I36,ID,7475.34,2018-08-20 18:25:00,2018-08-21 05:00:00,Monday,Tuesday
1,SUMMER,1K,A5,GRU,EZE,II,1710.75,2018-08-24 18:05:00,2018-08-24 21:05:00,Friday,Friday
2,SUMMER,1K,E5A,2L2,I36,ID,3952.66,2018-08-25 06:45:00,2018-08-25 09:50:00,Saturday,Saturday
3,SUMMER,1K,K0N,LED,I36,ID,2041.55,2018-08-26 15:15:00,2018-08-26 18:45:00,Sunday,Sunday
4,SUMMER,1K,AK6,32W,RUH,DI,2390.30,2018-08-22 21:25:00,2018-08-23 01:30:00,Wednesday,Thursday
5,SUMMER,1K,E,I36,JF8,DI,8012.26,2018-08-22 06:45:00,2018-08-22 10:50:00,Wednesday,Wednesday
6,SUMMER,1K,9,I36,L2X,DI,11034.38,2018-08-24 13:00:00,2018-08-24 16:50:00,Friday,Friday
7,SUMMER,1K,A8,YYZ,I36,ID,8167.30,2018-08-20 22:30:00,2018-08-21 15:15:00,Monday,Tuesday
8,SUMMER,1K,EN,26L,I36,ID,9243.97,2018-08-24 22:25:00,2018-08-25 16:40:00,Friday,Saturday
9,SUMMER,1K,6,ORD,I36,ID,8789.40,2018-08-24 21:40:00,2018-08-25 16:15:00,Friday,Saturday


In [134]:
# We will only consider the flights where arrival or departure airport is "I36" and where the flight direction is "ID" or "DI"
tariff = tariff[((tariff["DEP_APT"] == 'I36') | (tariff["ARR_APT"] == 'I36')  ) & (tariff["FLIGHT_DIRECTION"] != 'DD')]


### Inputs

In [136]:
season = input("season ")
flightno1 = input("flightno1 ")
hubarrive = int(input("hubarrive "))
flightno2 = input("flightno2 ")
hubdeparture = int(input("hubdeparture "))

season SUMMER
flightno1 N
hubarrive 3
flightno2 A
hubdeparture 3


 When "SEASON" and "FLIGHT NO" are given the following information can be extracted uniquely for each day: 

### OUTBOUND LIST
Outbound lit will consist of flights where departure airport is 'I36'( equivalently where flight direction is "DI"

In [137]:
outbound = tariff

In [138]:
outbound = outbound[(outbound["SEASON"] == season) & (outbound["FLIGHT_NO"] == flightno1)]

In [139]:
outbound.head()

,SEASON,CARRIER,FLIGHT_NO,DEP_APT,ARR_APT,FLIGHT_DIRECTION,DISTANCE,DEP_TIME,ARR_TIME,DEPARTURE_DAY,ARRIVAL_DAY
24,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-20 19:05:00,2018-08-21 11:55:00,Monday,Tuesday
54,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-26 19:05:00,2018-08-27 11:55:00,Sunday,Monday
88,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-24 19:05:00,2018-08-25 11:55:00,Friday,Saturday
97,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-22 19:05:00,2018-08-23 11:55:00,Wednesday,Thursday
99,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-23 19:05:00,2018-08-24 11:55:00,Thursday,Friday


In [140]:
outbound['ARR_TIME'] = outbound['ARR_TIME'] + dt.timedelta(hours=hubarrive)

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [141]:
outbound = outbound.merge(tariff, how='left', left_on=['SEASON', 'ARR_APT'], right_on=['SEASON', 'DEP_APT'])

In [142]:
outbound.head()

,SEASON,CARRIER_x,FLIGHT_NO_x,DEP_APT_x,ARR_APT_x,FLIGHT_DIRECTION_x,DISTANCE_x,DEP_TIME_x,ARR_TIME_x,DEPARTURE_DAY_x,...,CARRIER_y,FLIGHT_NO_y,DEP_APT_y,ARR_APT_y,FLIGHT_DIRECTION_y,DISTANCE_y,DEP_TIME_y,ARR_TIME_y,DEPARTURE_DAY_y,ARRIVAL_DAY_y
0,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-20 19:05:00,2018-08-21 14:55:00,Monday,...,1K,E,I36,JF8,DI,8012.26,2018-08-22 06:45:00,2018-08-22 10:50:00,Wednesday,Wednesday
1,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-20 19:05:00,2018-08-21 14:55:00,Monday,...,1K,9,I36,L2X,DI,11034.38,2018-08-24 13:00:00,2018-08-24 16:50:00,Friday,Friday
2,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-20 19:05:00,2018-08-21 14:55:00,Monday,...,1K,AM,I36,YYZ,DI,8167.30,2018-08-22 14:15:00,2018-08-22 18:00:00,Wednesday,Wednesday
3,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-20 19:05:00,2018-08-21 14:55:00,Monday,...,1K,E8,I36,J74,DI,7475.34,2018-08-24 01:35:00,2018-08-24 10:25:00,Friday,Friday
4,SUMMER,1K,N,JF8,I36,ID,8012.26,2018-08-20 19:05:00,2018-08-21 14:55:00,Monday,...,1K,KM5,I36,3VX,DI,2855.80,2018-08-23 00:30:00,2018-08-23 06:45:00,Thursday,Thursday


In [143]:
inbound = tariff

In [144]:
inbound = inbound[(inbound["SEASON"] == season) & (inbound["FLIGHT_NO"] == flightno2)]

In [145]:
inbound['DEP_TIME'] = inbound['DEP_TIME'] + dt.timedelta(hours=hubdeparture)

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [146]:
inbound = tariff.merge(inbound, how='right', left_on=['SEASON', 'ARR_APT'], right_on=['SEASON', 'DEP_APT'])

In [147]:
inbound.head()

,SEASON,CARRIER_x,FLIGHT_NO_x,DEP_APT_x,ARR_APT_x,FLIGHT_DIRECTION_x,DISTANCE_x,DEP_TIME_x,ARR_TIME_x,DEPARTURE_DAY_x,...,CARRIER_y,FLIGHT_NO_y,DEP_APT_y,ARR_APT_y,FLIGHT_DIRECTION_y,DISTANCE_y,DEP_TIME_y,ARR_TIME_y,DEPARTURE_DAY_y,ARRIVAL_DAY_y
0,SUMMER,1K,KE,J74,I36,ID,7475.34,2018-08-20 18:25:00,2018-08-21 05:00:00,Monday,...,1K,A,I36,JF8,DI,8012.26,2018-08-25 16:30:00,2018-08-25 17:25:00,Saturday,Saturday
1,SUMMER,1K,E5A,2L2,I36,ID,3952.66,2018-08-25 06:45:00,2018-08-25 09:50:00,Saturday,...,1K,A,I36,JF8,DI,8012.26,2018-08-25 16:30:00,2018-08-25 17:25:00,Saturday,Saturday
2,SUMMER,1K,K0N,LED,I36,ID,2041.55,2018-08-26 15:15:00,2018-08-26 18:45:00,Sunday,...,1K,A,I36,JF8,DI,8012.26,2018-08-25 16:30:00,2018-08-25 17:25:00,Saturday,Saturday
3,SUMMER,1K,A8,YYZ,I36,ID,8167.30,2018-08-20 22:30:00,2018-08-21 15:15:00,Monday,...,1K,A,I36,JF8,DI,8012.26,2018-08-25 16:30:00,2018-08-25 17:25:00,Saturday,Saturday
4,SUMMER,1K,EN,26L,I36,ID,9243.97,2018-08-24 22:25:00,2018-08-25 16:40:00,Friday,...,1K,A,I36,JF8,DI,8012.26,2018-08-25 16:30:00,2018-08-25 17:25:00,Saturday,Saturday


In [148]:
#detour factor

detour = outbound.merge(dist, how='left', left_on=['DEP_APT_x', 'ARR_APT_y'], right_on=['ORIGIN', 'DESTINATION'])

In [149]:
detour2 = inbound.merge(dist, how='left', left_on=['DEP_APT_x', 'ARR_APT_y'], right_on=['ORIGIN', 'DESTINATION'])

In [150]:
detour["DETOUR_FACTOR"] = (detour.DISTANCE_x + detour.DISTANCE_y)/detour.DISTANCE
detour2["DETOUR_FACTOR"] = (detour2.DISTANCE_x + detour2.DISTANCE_y)/detour2.DISTANCE

In [151]:
detour = detour[detour["DETOUR_FACTOR"] <= 1.4]
detour2 = detour2[detour2["DETOUR_FACTOR"] <= 1.4]

In [152]:
detour = detour[(detour['DEP_TIME_y']-detour['ARR_TIME_x'] <= dt.timedelta(hours=6))
       & (detour['DEP_TIME_y']-detour['ARR_TIME_x'] >= dt.timedelta(hours=1))]

In [153]:
detour2 = detour2[(detour2['DEP_TIME_y']-detour2['ARR_TIME_x'] <= dt.timedelta(hours=6))
       & (detour2['DEP_TIME_y']-detour2['ARR_TIME_x'] >= dt.timedelta(hours=1))]

In [154]:
market.head()

,SEASON,ORIGIN,DESTINATION,PSGR_COUNT
0,SUMMER,227,YYZ,6
1,SUMMER,224,222,3
2,SUMMER,226,JF8,1
3,SUMMER,227,3RG,11
4,SUMMER,227,6RV,245


In [155]:
new = detour.merge(market, how='left', 
                   left_on=['SEASON', 'DEP_APT_x', 'ARR_APT_y'], 
                   right_on=['SEASON', 'ORIGIN', 'DESTINATION'])

In [156]:
new2 = detour2.merge(market, how='left', 
                   left_on=['SEASON', 'DEP_APT_x', 'ARR_APT_y'], 
                   right_on=['SEASON', 'ORIGIN', 'DESTINATION'])

In [158]:
new = new[['DEPARTURE_DAY_x', 'FLIGHT_NO_x', 'DEP_APT_x', 'ARR_APT_y', 'PSGR_COUNT']]

In [159]:
new2 = new2[['DEPARTURE_DAY_x', 'FLIGHT_NO_x', 'DEP_APT_x', 'ARR_APT_y', 'PSGR_COUNT']]

In [103]:
new.columns = ['Day_of_Week', 'Outbound_Flight_No', 'Outbound_DEP_APT', 'Outbound_ARR_APT', 'Market_PSGR_Count']
new2.columns = ['Day_of_Week', 'Inbound_Flight_No', 'Inbound_DEP_APT', 'Inbound_ARR_APT', 'Market_PSGR_Count']

In [169]:
round = new.merge(new2, how='inner', left_on = 'ARR_APT_y', right_on = 'DEP_APT_x')

In [170]:
round

,DEPARTURE_DAY_x_x,FLIGHT_NO_x_x,DEP_APT_x_x,ARR_APT_y_x,PSGR_COUNT_x,DEPARTURE_DAY_x_y,FLIGHT_NO_x_y,DEP_APT_x_y,ARR_APT_y_y,PSGR_COUNT_y
0,Monday,N,JF8,8HE,326,Saturday,KMN,8HE,JF8,294
1,Monday,N,JF8,8HE,326,Wednesday,KMN,8HE,JF8,294
2,Monday,N,JF8,8HE,326,Thursday,KMN,8HE,JF8,294
3,Monday,N,JF8,8HE,326,Sunday,KMN,8HE,JF8,294
4,Monday,N,JF8,8HE,326,Monday,KMN,8HE,JF8,294
...,...,...,...,...,...,...,...,...,...,...
1943,Saturday,N,JF8,V7O,3841,Wednesday,AKA0,V7O,JF8,3989
1944,Saturday,N,JF8,V7O,3841,Thursday,AKA0,V7O,JF8,3989
1945,Saturday,N,JF8,V7O,3841,Tuesday,AKA0,V7O,JF8,3989
1946,Saturday,N,JF8,83C,651,Tuesday,AN0A,83C,JF8,633
